In [26]:
# The following statement imports the NLTK package.
import nltk

# The following statement imports a class called PlaintextCorpusReader.
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk import sent_tokenize
import string
import re 

import tqdm as notebook_tqdm


In [27]:
# instantiate finBERT
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

In [28]:
import os

# loop through text file in Data
rootdir = 'Data'
filelist = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filename = os.path.join(subdir, file)
        filelist.append(filename)

filelist = filelist[1:3]
# to be changed to the full list
print(filelist)

['Data/BA_transcripts/2016-Jan-28-BA.N-138020839119-Transcript.txt', 'Data/BA_transcripts/2015-Apr-22-BA.N-140063175207-Transcript.txt']


In [29]:
def textPreprocessing(filename):
    # load text
    file = open(filename, 'rt')
    text = file.read()
    file.close()

    # tokenize into sentences
    tokens = sent_tokenize(text)

    # alpha to lowercase
    tokens = [w.lower() for w in tokens]

    # remove punctuation and whitespaces
    remove = string.punctuation
    remove = re.sub(r"[-]+", "", remove)
    pattern = r"[{}]".format(remove + '-')
    clean_list = []

    for sentence in tokens:
        sentence = re.sub(pattern, "", sentence) 
        sentence = sentence.split()
        sentence = ' '.join(sentence)
        clean_list.append(sentence)

    return clean_list

# no neeed to remove stopwords and conduct lemmatization
# may worsen prediction for transformer


In [30]:
# function that takes returns a document level sentiment score

def sentimentAnalysis(filename):
    clean_list = textPreprocessing(filename)
    # pipeline: API that wraps finBERT sentiment analysis 
    nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
    results = nlp(clean_list)
    
    negative = 0
    positive = 0
    for element in results:
        if element['label'] == "Negative":
            negative += 1
        elif element['label'] == "Positive":
            positive += 1
    total = len(results)

    sentiment_score = (positive - negative)/ total
    return sentiment_score

    # +ve sentiment score: positive earnings call
    # -ve sentiment score: negative earnings call

    # https://www.alpha-sense.com/blog/engineering/sentiment-score/
    # sentiment score: (positive - negative) / total number of statements





In [32]:
sentiment_score_dict = {'filename':[], "sentiment_score": []}
for filename in filelist:
    sentiment_score = sentimentAnalysis(filename)
    filename = filename.split('/')[-1]
    sentiment_score_dict["filename"].append(filename)
    sentiment_score_dict["sentiment_score"].append(sentiment_score)

print(sentiment_score_dict)


{'filename': ['2016-Jan-28-BA.N-138020839119-Transcript.txt', '2015-Apr-22-BA.N-140063175207-Transcript.txt'], 'sentiment_score': [0.29038112522686027, 0.1873873873873874]}


In [34]:
df = pd.DataFrame(sentiment_score_dict)
df.to_excel('sentiment_score.xlsx', index=False)

In [ ]:
# # stopword removal
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# words = [w for w in words if not w in stop_words]
# print(words[:100])

In [ ]:
# # lemmatization
# from nltk.stem import WordNetLemmatizer
# lemmatizer = WordNetLemmatizer()
# lemmatized = [lemmatizer.lemmatize(word) for word in words]
# print(lemmatized) 